<a href="https://colab.research.google.com/github/SharonCao0920/CloudComputing/blob/main/Machine_Learning/Movie_Recommendation_System/Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Step 1: Go to the correct directory**

In [6]:
%cd drive/MyDrive/Colab Notebooks/CS570
!pwd

/content/drive/MyDrive/Colab Notebooks/CS570
/content/drive/MyDrive/Colab Notebooks/CS570


### **Step 2: Convert the .txt data file to .csv file**

In [8]:
import pandas as pd
# covert txt to cvs
read_file = pd.read_csv ("movies.txt")
read_file.to_csv ("movies.csv", index=None)

read_file = pd.read_csv ("ratings.txt")
read_file.to_csv ("ratings.csv", index=None)

read_file = pd.read_csv ("tags.txt")
read_file.to_csv ("tags.csv", index=None)

### **Step 3: Install pyspark**

In [9]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### **Step 4: Import libraries**

In [10]:
import pandas as pd
from pyspark.sql.functions import col, explode
from pyspark import SparkContext

### **Step 5: Initiate spark session**

In [11]:
from pyspark.sql import SparkSession
sc = SparkContext
# sc.setCheckpointDir('checkpoint')
spark = SparkSession.builder.appName('Recommendations').getOrCreate()

### **Step 6: Load data**

In [19]:
%cd drive/MyDrive/Colab Notebooks/CS570
movies = spark.read.csv("./drive/MyDrive/Colab Notebooks/CS570/movies.csv",header=True)
ratings = spark.read.csv("./drive/MyDrive/Colab Notebooks/CS570/ratings.csv",header=True)

[Errno 2] No such file or directory: 'drive/MyDrive/Colab Notebooks/CS570'
/content/drive/MyDrive/Colab Notebooks/CS570


In [20]:
ratings.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows



In [21]:
ratings.printSchema()

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [22]:
ratings = ratings.\
    withColumn('userId', col('userId').cast('integer')).\
    withColumn('movieId', col('movieId').cast('integer')).\
    withColumn('rating', col('rating').cast('float')).\
    drop('timestamp')
ratings.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|      6|   4.0|
|     1|     47|   5.0|
|     1|     50|   5.0|
|     1|     70|   3.0|
|     1|    101|   5.0|
|     1|    110|   4.0|
|     1|    151|   5.0|
|     1|    157|   5.0|
|     1|    163|   5.0|
|     1|    216|   5.0|
|     1|    223|   3.0|
|     1|    231|   5.0|
|     1|    235|   4.0|
|     1|    260|   5.0|
|     1|    296|   3.0|
|     1|    316|   3.0|
|     1|    333|   5.0|
|     1|    349|   4.0|
+------+-------+------+
only showing top 20 rows



### **Step 7: Calculate sparsity**

In [23]:
# Count the total number of ratings in the dataset
numerator = ratings.select("rating").count()

# Count the number of distinct userIds and distinct movieIds
num_users = ratings.select("userId").distinct().count()
num_movies = ratings.select("movieId").distinct().count()

# Set the denominator equal to the number of users multiplied by the number of movies
denominator = num_users * num_movies

# Divide the numerator by the denominator
sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("The ratings dataframe is ", "%.2f" % sparsity + "% empty.")

The ratings dataframe is  98.30% empty.


### **Step 8: Interpret ratings**

In [24]:
# Group data by userId, count ratings
userId_ratings = ratings.groupBy("userId").count().orderBy('count', ascending=False)
userId_ratings.show()

+------+-----+
|userId|count|
+------+-----+
|   414| 2698|
|   599| 2478|
|   474| 2108|
|   448| 1864|
|   274| 1346|
|   610| 1302|
|    68| 1260|
|   380| 1218|
|   606| 1115|
|   288| 1055|
|   249| 1046|
|   387| 1027|
|   182|  977|
|   307|  975|
|   603|  943|
|   298|  939|
|   177|  904|
|   318|  879|
|   232|  862|
|   480|  836|
+------+-----+
only showing top 20 rows



In [25]:
# Group data by userId, count ratings
movieId_ratings = ratings.groupBy("movieId").count().orderBy('count', ascending=False)
movieId_ratings.show()

+-------+-----+
|movieId|count|
+-------+-----+
|    356|  329|
|    318|  317|
|    296|  307|
|    593|  279|
|   2571|  278|
|    260|  251|
|    480|  238|
|    110|  237|
|    589|  224|
|    527|  220|
|   2959|  218|
|      1|  215|
|   1196|  211|
|   2858|  204|
|     50|  204|
|     47|  203|
|    780|  202|
|    150|  201|
|   1198|  200|
|   4993|  198|
+-------+-----+
only showing top 20 rows



### **Step 9: Build Out An ALS Model**

In [26]:
# Import the required functions
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [39]:
# Create test and train set
(train, test) = ratings.randomSplit([0.8, 0.2], seed = 1234)

# Create ALS model
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", nonnegative = 
          True, implicitPrefs = False, coldStartStrategy="drop")

# Confirm that a model called "als" was created
type(als)

pyspark.ml.recommendation.ALS

### **Step 10: Tell Spark how to tune your ALS model**

In [50]:
# Import the requisite items
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Add hyperparameters and their respective values to param_grid
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 50, 100, 150]) \
            .addGrid(als.regParam, [.01, .05, .1, .15]) \
            .build()
            #             .addGrid(als.maxIter, [5, 50, 100, 200]) \

           
# Define evaluator as RMSE and print length of evaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction") 
print ("Num models to be tested: ", len(param_grid))

Num models to be tested:  16


### **Step 11: Build your cross validation pipeline**

In [51]:
# Build cross validation using CrossValidator
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)

# Confirm cv was built
print(cv)

CrossValidator_38a358787dbc


### **Step 12: Best Model and Best Model Parameters**

In [52]:
#Fit cross validator to the 'train' dataset
model = cv.fit(train)
#Extract best model from the cv model above
best_model = model.bestModel


In [53]:
# # Print best_model
# print(type(best_model))

# Complete the code below to extract the ALS model parameters
print("**Best Model**")

# # Print "Rank"
print("  Rank:", best_model._java_obj.parent().getRank())

# Print "MaxIter"
print("  MaxIter:", best_model._java_obj.parent().getMaxIter())

# Print "RegParam"
print("  RegParam:", best_model._java_obj.parent().getRegParam())

**Best Model**
  Rank: 50
  MaxIter: 10
  RegParam: 0.15


In [54]:
# View the predictions
test_predictions = best_model.transform(test)
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)

0.8694309662864149


In [55]:
test_predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   148|    356|   4.0| 3.4979415|
|   148|   4896|   4.0| 3.5384233|
|   148|   4993|   3.0| 3.4743285|
|   148|   7153|   3.0| 3.4075155|
|   148|   8368|   4.0|  3.621026|
|   148|  40629|   5.0| 3.2884903|
|   148|  50872|   3.0| 3.6388183|
|   148|  60069|   4.5| 3.6809375|
|   148|  69757|   3.5|  3.332219|
|   148|  72998|   4.0|  3.287846|
|   148|  81847|   4.5| 3.4478338|
|   148|  98491|   5.0|  3.708793|
|   148| 115617|   3.5| 3.6338181|
|   148| 122886|   3.5|  3.441301|
|   463|    296|   4.0|  4.170789|
|   463|    527|   4.0|  3.759904|
|   463|   2019|   4.0|  3.908107|
|   471|    527|   4.5|  3.716461|
|   471|   6016|   4.0| 3.9353473|
|   471|   6333|   2.5| 3.1815176|
+------+-------+------+----------+
only showing top 20 rows



### **Step 13: Make Recommendations**

In [56]:
# Generate n Recommendations for all users
nrecommendations = best_model.recommendForAllUsers(10)
nrecommendations.limit(10).show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|     1|[{3379, 5.7534437...|
|     3|[{5746, 4.8547745...|
|     5|[{3379, 4.5763803...|
|     6|[{33649, 4.747345...|
|     9|[{3379, 4.92165},...|
|    12|[{67618, 5.722233...|
|    13|[{3379, 4.9300547...|
|    15|[{3379, 4.483718}...|
|    16|[{3379, 4.6229935...|
|    17|[{3379, 5.2144084...|
+------+--------------------+



In [57]:
nrecommendations = nrecommendations\
    .withColumn("rec_exp", explode("recommendations"))\
    .select('userId', col("rec_exp.movieId"), col("rec_exp.rating"))

nrecommendations.limit(10).show()

+------+-------+---------+
|userId|movieId|   rating|
+------+-------+---------+
|     1|   3379|5.7534437|
|     1|  33649|5.5914445|
|     1|   5490|5.5728765|
|     1| 171495|5.4150558|
|     1|   5915| 5.379195|
|     1|  78836|5.3676977|
|     1|   7071| 5.344718|
|     1|  26073| 5.344718|
|     1| 179135| 5.344718|
|     1| 117531| 5.344718|
+------+-------+---------+



## **Do the recommendations make sense?**

In [58]:
nrecommendations.join(movies, on='movieId').filter('userId = 100').show()

+-------+------+---------+--------------------+--------------------+
|movieId|userId|   rating|               title|              genres|
+-------+------+---------+--------------------+--------------------+
|  67618|   100|5.1550994|Strictly Sexual (...|Comedy|Drama|Romance|
|   3379|   100| 5.104017| On the Beach (1959)|               Drama|
|  33649|   100| 5.086438|  Saving Face (2004)|Comedy|Drama|Romance|
|  26073|   100|4.9374614|Human Condition I...|           Drama|War|
|  84273|   100|4.9374614|Zeitgeist: Moving...|         Documentary|
| 179135|   100|4.9374614|Blue Planet II (2...|         Documentary|
| 117531|   100|4.9374614|    Watermark (2014)|         Documentary|
|  74226|   100|4.9374614|Dream of Light (a...|   Documentary|Drama|
| 184245|   100|4.9374614|De platte jungle ...|         Documentary|
|   7071|   100|4.9374614|Woman Under the I...|               Drama|
+-------+------+---------+--------------------+--------------------+



In [59]:
ratings.join(movies, on='movieId').filter('userId = 100').sort('rating', ascending=False).limit(10).show()

+-------+------+------+--------------------+--------------------+
|movieId|userId|rating|               title|              genres|
+-------+------+------+--------------------+--------------------+
|   1101|   100|   5.0|      Top Gun (1986)|      Action|Romance|
|   1958|   100|   5.0|Terms of Endearme...|        Comedy|Drama|
|   2423|   100|   5.0|Christmas Vacatio...|              Comedy|
|   4041|   100|   5.0|Officer and a Gen...|       Drama|Romance|
|   5620|   100|   5.0|Sweet Home Alabam...|      Comedy|Romance|
|    368|   100|   4.5|     Maverick (1994)|Adventure|Comedy|...|
|    934|   100|   4.5|Father of the Bri...|              Comedy|
|    539|   100|   4.5|Sleepless in Seat...|Comedy|Drama|Romance|
|     16|   100|   4.5|       Casino (1995)|         Crime|Drama|
|    553|   100|   4.5|    Tombstone (1993)|Action|Drama|Western|
+-------+------+------+--------------------+--------------------+

